In [32]:
import numpy as np
import gym
import random

In [33]:
ENV_NAME = "Taxi-v3"
env = gym.make(ENV_NAME)
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [34]:
print("Number of actions: %d" % env.action_space.n)
print("Number of states: %d" % env.observation_space.n)

Number of actions: 6
Number of states: 500


In [35]:
action_size = env.action_space.n
state_size = env.observation_space.n

np.random.seed(123)
env.seed(123)

[123]

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape
from keras.optimizer_v1 import Adam
Adam._name = 'hey'

In [37]:
env.reset()
env.step(env.action_space.sample())[0]

241

In [38]:
import keras
model = keras.Sequential()
model.add(Embedding(500, 10, input_length=1))
model.add(Reshape((10,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(action_size, activation='linear'))
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1, 10)             5000      
                                                                 
 reshape_5 (Reshape)         (None, 10)                0         
                                                                 
 dense_12 (Dense)            (None, 50)                550       
                                                                 
 dense_13 (Dense)            (None, 50)                2550      
                                                                 
 dense_14 (Dense)            (None, 50)                2550      
                                                                 
 dense_15 (Dense)            (None, 6)                 306       
                                                                 
Total params: 10,956
Trainable params: 10,956
Non-trai

In [39]:
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=action_size, memory=memory, nb_steps_warmup=500, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=1000000, visualize=False, verbose=1, nb_max_episode_steps=99, log_interval=100000)

Training for 1000000 steps ...
Interval 1 (0 steps performed)
100000/100000 [==============================] - 1727s 17ms/step - reward: -1.3054
1148 episodes - episode_reward: -113.655 [-855.000, 15.000] - loss: 3.821 - mae: 24.304 - mean_q: -23.987 - prob: 1.000

Interval 2 (100000 steps performed)
100000/100000 [==============================] - 1842s 18ms/step - reward: -0.9097
2460 episodes - episode_reward: -37.005 [-486.000, 15.000] - loss: 2.979 - mae: 18.985 - mean_q: -12.820 - prob: 1.000

Interval 3 (200000 steps performed)
100000/100000 [==============================] - 1898s 19ms/step - reward: 0.1504
6764 episodes - episode_reward: 2.223 [-135.000, 15.000] - loss: 0.104 - mae: 7.167 - mean_q: 12.262 - prob: 1.000

Interval 4 (300000 steps performed)
100000/100000 [==============================] - 1382s 14ms/step - reward: 0.1533
6757 episodes - episode_reward: 2.269 [-100.000, 15.000] - loss: 0.002 - mae: 7.364 - mean_q: 12.694 - prob: 1.000

Interval 5 (400000 steps pe

In [40]:
dqn.test(env, nb_episodes=5, visualize=True, nb_max_episode_steps=99)

Testing for 5 episodes ...
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : :_|
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |

In [41]:
dqn.save_weights('dqn_{}_weights.h5f'.format("Taxi-v3"), overwrite=True)